In [1]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import re
import xml.etree.ElementTree as ET
import xmltodict
import collections
import itertools
from ratelimit import limits, sleep_and_retry
# Import API key

In [2]:
StateSelected = ['GA']

In [3]:
# C:\Users\cuong\Downloads\free-zipcode-database-Primary.csv
zipcodes=pd.read_csv('C:/Users/cuong/Downloads/free-zipcode-database-Primary.csv')
zipcodes.head(10)

,Zipcode,State
0,705,PR
1,610,PR
2,611,PR
3,612,PR
4,601,PR
5,631,PR
6,602,PR
7,603,PR
8,604,PR
9,605,PR


In [4]:
gazipcodes= zipcodes[zipcodes.State.isin(StateSelected)]

In [5]:
gazipcodes=gazipcodes
len(gazipcodes)

973

## Perform API Calls

In [6]:

#conn = http.client.HTTPSConnection("search.onboard-apis.com") 

params = { 
    'accept': "application/json", 
    'apikey': "9d078487e223b1c4d54c3f3a3f628803", 
} 

In [ ]:
FullAddress=[]
Street=[]
CityStateZip=[]
Latitude=[]
Longitude=[]
pd.Series(FullAddress)
pd.Series(Latitude)
pd.Series(Longitude)
Data_set=pd.DataFrame({'FullAddress':pd.Series(FullAddress),'Street':pd.Series(Street),'CityStateZip':pd.Series(CityStateZip),
                       'Latitude':pd.Series(Latitude),'Longitude':pd.Series(Longitude)})
url=f"https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?postalcode=postalcode&page=1&pagesize=100"
for index, row in gazipcodes.iterrows():
    postalcode=row['Zipcode']
    url=f"https://search.onboard-apis.com/propertyapi/v1.0.0/property/address?postalcode={postalcode}&page=1&pagesize=1000"
    res = requests.get(url,headers=params)
    data =json.loads(res.text)
    we=data
    try:
        d=pd.DataFrame(data['property'])
        d['FullAddress']=d['address'].apply(lambda a:a['oneLine'])
        d['Street']=d['address'].apply(lambda d:d['line1'])
        d['CityStateZip']=d['address'].apply(lambda d:d['line2'])
        d['Latitude']=d['location'].apply(lambda c:c['latitude'])
        d['Longitude']=d['location'].apply(lambda b:b['longitude'])
        datatoappend=d[['FullAddress','Street','CityStateZip','Latitude','Longitude']]
        Data_set=Data_set.append(datatoappend)
             
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [ ]:
Data_set.head()

In [ ]:
Data_set['Zilliw format add']=Data_set['Street'].replace(" ", "+",regex=True)
Data_set['Zilliw format Zipciy']=Data_set['CityStateZip'].replace(" ", "+",regex=True)


In [ ]:
Data_set.to_csv('data.csv')

In [ ]:
#Next is to feed the addresess to zillow api
# OpenWeatherMap API Key
zid = 'X1-ZWz18cayvld3pn_8gte4'
#api_key = api_keys.api_key

# Starting URL for Weather Map API Call
#url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key 

#url = f"http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={api_key}&address=109 TAYLOR BRIDGE RD SW&citystatezip=CALHOUN GA 30701"  
#url = f"http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={api_key}&address=5888+Colonnade+Drive&citystatezip=Rex,+GA+30273"  


In [ ]:

# #url = f"http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={api_key}&address=5888+Colonnade+
# Drive&citystatezip=Rex,+GA+30273"
# url2 = f"http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={zid}&address={params2}"
w=[]
for index, row in Data_set.iterrows():
    # get city, lat, lng from df
    # change location each iteration 
    add=row['Zilliw format add']
    statezip=row['Zilliw format Zipciy']
    
    # Use the search term: "International Airport" and our lat/lng
    params2=f'{add}&citystatezip={statezip}'
    url2 = f"http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={zid}&address={params2}"

    # make request and print url
    response2 = requests.get(url2, params=params2)
    we       = response2.content
    w.append(we)
    we=[]
    time.sleep(2)


In [ ]:
zpid=[]
zest=[]
add=[]
city=[]
zipC=[]
lat=[]
lon=[]
type=[]
rooms=[]
sqft=[]
state=[]
lastsoldprice=[]
year_built=[]
for i,k in enumerate(w):
    tree=ET.fromstring(w[i])
    results=tree.findall('./response/results/result')
    if len(results)!=0:
     re=results[0] 
     dic=xmltodict.parse(ET.tostring(re))
    try:
         Zid=dic['result'].get('zpid',"none")
         zestimate=dic['result']['zestimate']['amount'].get('#text',"none")
         st=dic['result']['address'].get('street','none')
         zip=dic['result']['address'].get('zipcode','none')
         cit=dic['result']['address'].get('city','none')
         price=dic['result']['lastSoldPrice'].get('#text',"none")
         room=dic['result'].get('bedrooms','None')
         usecode=dic['result'].get('useCode',"none")
         sqftt=dic['result'].get('finishedSqFt','None')
         yearbuilt=dic['result'].get('yearBuilt',"none")
#     sq=dic['result']['finishedSqFt']
#     year=dic['result']['yearBuilt']
         year_built.append(yearbuilt)
         zest.append(zestimate)
         zpid.append(Zid)
         add.append(st)
         zipC.append(zip)
         city.append(cit)
         lastsoldprice.append(price)
         rooms.append(room)
         type.append(usecode)
         sqft.append(sqftt)
         zestimate=[]
         Zid=[]
         price=[]
         room=[]
         usecode=[]
         sqftt=[]
         cit=[]
         usecode=[]
         yearbuilt=[]
    except (KeyError, IndexError):
        print('Skiping next')
        
   

In [ ]:
dataneed=pd.DataFrame({'Zillow Id':pd.Series(zpid),'Zestimate':pd.Series(zest),'Street':pd.Series(add),
                       'City':pd.Series(city),'ZipCode':pd.Series(zipC),'lastSold':pd.Series(lastsoldprice),
                       'totalrooms':pd.Series(rooms),'typeofhome':pd.Series(type),'Squareft':pd.Series(sqft),
                       'YearBuilt':pd.Series(year_built)})

In [ ]:
dataneed.drop_duplicates()